# Sentinel 2 Composites
Version 20230713d includes imagery inputs from 2019 through the end of the 2023 growing season

Imagery is tiled in the 50km AKVEG tiles (5000x5000 pixels each at 10 m resolution) and stored in a Google Cloud Storage bucket (akveg-data/s2_sr_2019_2023_gMedian_v20240713d) as part of the akveg-map project. The data can be accessed as a mosaic within Google Earth Engine via as a cloud-backed image collection (https://developers.google.com/earth-engine/Earth_Engine_asset_from_cloud_geotiff)

## Version history

### v20240713d
GEE snapshot: [https://code.earthengine.google.com/ecb8bb60b4985e1cd1a03865002c6992]
GEE script path: [https://code.earthengine.google.com/?scriptPath=users%2Fmmacander%2Fakveg_map%3Asentinel_2%2Fs2_medians_v20240713d]

As of 2024-07-17, the lastest version of the Sentinel 2 reflectance composites in v20240713d. A description is below.

Sentinel 2 Level 2A (surface reflectance) data ([https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED]) are available for Alaska and Canada from 2019 through the present. The product is produced by the European Space Agency (ESA) and incorporates both an atmospheric correction and an illumination correction (unlike Landsat Collection 2 surface reflectance, for example, which incorporates an atmospheric correction only). The DEM used for the illumination correction was changed during winter 2020-2021 to the Copernicus GLO-30 global DEM ([https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_DEM_GLO30]), and the more recent results based on GLO-30 appear to be improved overall and have fewer artifacts at abrupt terrain breaks compared to the earlier results. 

Input data are first screened at the granule level, to include only images that meet criteria of 1) years between 2019 and 2023 inclusive; 2) day of year >= the first day of year when the maximum solar elevation is 40 degrees or higher; 3) day of year <= the last day of year when the maximum solar elevation is 25 degrees or lower; and 4) a granule 'cloudy pixel percentage' of 80% or less. For the day of year criteria, the thresholds are assigned at the level of the 50 km output tile.

The images are then masked to exclude clouds, cloud shadow, snow, and other bad data. 1) Clouds and cloud shadow are masked based on the Cloud Score+ algorithm ([https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_CLOUD_SCORE_PLUS_V1_S2_HARMONIZED]), masking pixels where the 'cs' band < 0.6. 2) Snow and other bad data are masked based on the scene classification map ('SCL' band of the image), masking values of 1 (saturated or defective), 2 (dark area pixels), 3 (cloud shadow), and 11 (snow / ice). 3) For scenes where the Dynamic World ([https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_DYNAMICWORLD_V1]) product was generated, i.e. any scene where the 'cloudy pixel percentage' was <= 35%, pixels where the labelled class was 9 (snow/ice) are also masked.

After masking, the Sentinel 2 data were mosaicked based on date and orbit number, to flatten data from overlap areas that occur at the edges of MGRS tiles. Without this step, there would be duplicate data from the overlap areas. 

5 seasonal composites are generated from the quality-masked images, nominally representing spring, early summer, midsummer, late summer, and fall. Since snow-masking is applied, the spring and fall periods are intended to represent the snow-free portion of those periods. The seasons are defined based on an existing per-pixel analysis of the dates of snow-free Sentinel 2 imagery ([https://code.earthengine.google.com/6b223ec1a1419a7b8210900b23f3e4c3] or [https://code.earthengine.google.com/?scriptPath=users%2Fmmacander%2Fakveg_map%3Asentinel_2%2Fs2_v20230113_doys]) that included 2019–2022 imagery filtered and masked similarly to the rules above. Day of year percentiles were calculated for each pixel based on the dates of snow-free observations. The central dates of the composite periods were defined as follows: 1) spring = 5th percentile of the snow-free observation dates; 2) midsummer = July 31; 3) fall = 95th percentile of the snow-free observation dates; 4) early summer = 25% of the way between spring and midsummer; and 5) late summer = midway between midsummer and fall. 

The composites were then calculated as the geometric median of observations within a time window around the central date of each season. For each season, both a narrow and wide time window were considered; if the number of quality-masked observations within the narrow time window was < 3, then the wide time window was used. If the number of quality-masked observations with the wide time window was < 3, then alternate seasonal windows were considered, as described below. The general assumption behind this approach was that the geometric median was not a robust metric with less than 3 observations, because if one of the the observations was an outlier then the result would be heavily influenced by the outlier. The exception was midsummer with a wide time window, for which only one quality-masked observation was required. This was to help ensure that every pixel had some reflectance estimate for each seasonal window (midsummer was a backup for each of the other seasonal windows). Generally, the minimum number of observations was easily achieved for the wide window in each seasonal period, except for snow-dominated areas (on or near glaciers and snow-fields) and some geographic regions with high cloud cover and low observation density (e.g. portions of the Aleutian Islands).

TODO: Insert tables with composite windows (narrow and wide), including tier ID. Insert table with backup tiers.

TODO: Consider using DOY analysis from the complete Landsat TM, ETM+, and OLI record (1984–present), excluding Landsat 7 ETM+ data collected after the SLC-off malfunction to avoid striping artifacts. And/or, the refined snow analysis based on the Sentinel 2 Dynamic World product.

TODO: Consider restoring fire masking. Fire masking limited observations to pre- and/or post-fire imagery for pixels where fires occurred during the 2019-2023 window. Pre- and post-fire products can be produced later as separate tiled product, only processing tiles that intersect fire perimeters, and only for pixels within fire perimeter (could consider a buffer). In main composites, when combined with fallback tiers, fire masking was leading to some composites with mix of pre- and post-fire in different seasons and was contributing to high compute load.

TODO: Consider selective use of a full growing season backup tier (e.g. ~May 1-October 31). It was computationally intensive and was only needed for a small number of pixels, mostly in permanent snowfields. Was contributing to very high computer and/or processing failures. Consider reviewing areas with limited data and running full season or something similar there (e.g. mid-Aleutians).

### v2024-03-11
GEE snapshot: [https://code.earthengine.google.com/f7ccb6d08f9f201c2f09d6433719433e]
GEE script path: [https://code.earthengine.google.com/?scriptPath=users%2Fmmacander%2Fakveg_map%3Asentinel_2%2Fs2_medians_v20240311]

Prior version for portion of study area. Demonstrates much reduced snow contamination in early season composites compared to previous version. Also demonstrates partial fire masking, using fire polygons for 2019-2023 to mask input imagery that was acquired the year of or before a fire. So where a 2020 fire is mapped in the fire perimeter polygon data, only 2021-2023 data is included in the composites. This should make the composites represent the current condition (after the fire). Problems with this approach are that for training, we may prefer the pre-fire conditions. Also, fires in 2023 will have no image because it is all masked. The script currently falls back to a non-seasonal full snow-free season composite when there is insufficient data available but this is not desirable behavior for 2023 firescars.

### v2023-04-18
GEE snapshot: [https://code.earthengine.google.com/4ab67b271cae54051b8d756d847c2a95]
GEE script path: [https://code.earthengine.google.com/?scriptPath=users%2Fmmacander%2Fakveg_map%3Asentinel_2%2Fs2_medians_v20230418]

Prior version for complete study area. Less robust snow masking. Only a single time window per composite period (no narrow and wide windows). No backstops to fill in composites periods. Partial fire masking.

## Visualization
Visualize results after creating the Cloud GeoTiff Backed Earth Engine Assets with this viz script:

[https://code.earthengine.google.com/?scriptPath=users%2Fmmacander%2Fakveg_map%3Asentinel_2%2Fs2_viz]



## Background on Cloud GeoTiff Backed Earth Engine Assets

***Note:*** *The REST API contains new and advanced features that may not be suitable for all users.  If you are new to Earth Engine, please get started with the [JavaScript guide](https://developers.google.com/earth-engine/guides/getstarted).*

Earth Engine can load images from Cloud Optimized GeoTiffs (COGs) in Google Cloud Storage ([learn more](https://developers.google.com/earth-engine/guides/image_overview#images-from-cloud-geotiffs)).  This notebook demonstrates how to create Earth Engine assets backed by COGs.  An advantage of COG-backed assets is that the spatial and metadata fields of the image will be indexed at asset creation time, making the image more performant in collections.  (In contrast, an image created through `ee.Image.loadGeoTIFF` and put into a collection will require a read of the GeoTiff for filtering operations on the collection.)  A disadvantage of COG-backed assets is that they may be several times slower than standard assets when used in computations.

To create a COG-backed asset, make a `POST` request to the Earth Engine [`CreateAsset` endpoint](https://developers.google.com/earth-engine/reference/rest/v1alpha/projects.assets/create).  As shown in the following, this request must be authorized to create an asset in your user folder.

## Start an authorized session

To be able to make an Earth Engine asset in your user folder, you need to be able to authenticate as yourself when you make the request.  You can use credentials from the Earth Engine authenticator to start an [`AuthorizedSession`](https://google-auth.readthedocs.io/en/master/reference/google.auth.transport.requests.html#google.auth.transport.requests.AuthorizedSession).  You can then use the `AuthorizedSession` to send requests to Earth Engine.

In [7]:
import ee
from google.auth.transport.requests import AuthorizedSession

ee.Authenticate()  #  or !earthengine authenticate --auth_mode=gcloud
session = AuthorizedSession(ee.data.get_persistent_credentials())

## Create list of cloud geotiffs in bucket and image collection

Create list of cogs in a bucket folder to load into an imageCollection.

Create list of cogs that have already been loaded into the imageCollection.

Run in bash in a conda env with gsutil and earthengine command line installed and authenticated.

TODO: Configure it to run directly in python

```
cd /data/gis/gis_projects/2024/24-224_Land_Cover_Metrics_Susitna_Wolf/sentinel2_gMedian

gsutil ls gs://akveg-data/s2_sr_2019_2023_gMedian_v20240713d/*.tif > s2_cogs_v20240713d.txt

earthengine ls projects/akveg-map/assets/s2_sr_2019_2023_gMedian_v20240713d > s2_cogs_v20240713d_inColl.txt
```

In [32]:
import pandas
import os

cogList_txt = '/data/gis/gis_projects/2024/24-224_Land_Cover_Metrics_Susitna_Wolf/sentinel2_gMedian/s2_cogs_v20240713d.txt'
cogListIC_txt = '/data/gis/gis_projects/2024/24-224_Land_Cover_Metrics_Susitna_Wolf/sentinel2_gMedian/s2_cogs_v20240713d_inColl.txt'

# Function to read a text file and return a list of paths
def read_paths_from_file(filepath):
    with open(filepath, 'r') as file:
        paths = file.read().splitlines()
    return paths

# Function to extract the last part of the path and remove the extension
def get_filename_without_extension(path):
    return os.path.splitext(os.path.basename(path))[0]

cogList = read_paths_from_file(cogList_txt)
cogListIC = read_paths_from_file(cogListIC_txt)

# Function to save a list of paths to a text file
def save_paths_to_file(paths, filepath):
    with open(filepath, 'w') as file:
        for path in paths:
            file.write(f"{path}\n")

# Apply the function to both lists
filenames1 = [get_filename_without_extension(path) for path in cogList]
filenames2 = [get_filename_without_extension(path) for path in cogListIC]

# Convert lists to pandas Series
filenames_series1 = pandas.Series(filenames1, index=cogList)
filenames_series2 = pandas.Series(filenames2, index=cogListIC)

# Find filenames in list1 that are not in list2
unique_to_list1 = filenames_series1[~filenames_series1.isin(filenames2)]

# Print the results
# print("List of names without extensions from list1:")
# print(filenames1)

# print("\nList of names without extensions from list2:")
# print(filenames2)

print("\nPaths in list1 that are not in list2 based on filenames:")
print(unique_to_list1.index.tolist()[:5])
print(len(unique_to_list1.index.tolist()))

save_paths_to_file(unique_to_list1.index.tolist(), '/data/gis/gis_projects/2024/24-224_Land_Cover_Metrics_Susitna_Wolf/sentinel2_gMedian/s2_cogs_v20240713d_notInColl.txt')




Paths in list1 that are not in list2 based on filenames:
['gs://akveg-data/s2_sr_2019_2023_gMedian_v20240713d/s2_sr_2019_2023_gMedian_AK050H60V10_all_v20240713d.tif', 'gs://akveg-data/s2_sr_2019_2023_gMedian_v20240713d/s2_sr_2019_2023_gMedian_AK050H60V11_all_v20240713d.tif', 'gs://akveg-data/s2_sr_2019_2023_gMedian_v20240713d/s2_sr_2019_2023_gMedian_AK050H60V12_all_v20240713d.tif', 'gs://akveg-data/s2_sr_2019_2023_gMedian_v20240713d/s2_sr_2019_2023_gMedian_AK050H60V13_all_v20240713d.tif', 'gs://akveg-data/s2_sr_2019_2023_gMedian_v20240713d/s2_sr_2019_2023_gMedian_AK050H60V14_all_v20240713d.tif']
61


## Open list of geotiffs to ingest

In [33]:
import pandas
# cogs = pandas.read_csv('/data/gis/gis_projects/2024/24-224_Land_Cover_Metrics_Susitna_Wolf/sentinel2_gMedian/s2_cogs_v20240710.txt', header=None,names=['tif'])
# cogs = pandas.read_csv('/data/gis/gis_projects/2024/24-224_Land_Cover_Metrics_Susitna_Wolf/sentinel2_gMedian/s2_cogs_v20240713d.txt', header=None,names=['tif'])
cogs = pandas.read_csv('/data/gis/gis_projects/2024/24-224_Land_Cover_Metrics_Susitna_Wolf/sentinel2_gMedian/s2_cogs_v20240713d_notInColl.txt', header=None,names=['tif'])
print(cogs[0:2])
print(len(cogs.index))


                                                 tif
0  gs://akveg-data/s2_sr_2019_2023_gMedian_v20240...
1  gs://akveg-data/s2_sr_2019_2023_gMedian_v20240...
61


## Setup parameters

In [34]:
import json
# from urllib.parse import urlparse
import os
from pprint import pprint

# Earth Engine enabled Cloud Project.
project_folder = 'akveg-map'
collection = 's2_sr_2019_2023_geometricMedian_v20240713d'

## Create empty image collection as target
TODO Automate creation of empty image collection.

For now, manually create empty image collection with earthengine CLI

```
earthengine create collection projects/akveg-map/assets/s2_sr_2019_2023_gMedian_v20240713d
```

## View list of cogs to ingest
Optional, can skip when list is long.

In [35]:
for cog in cogs['tif']:
    print(cog)

gs://akveg-data/s2_sr_2019_2023_gMedian_v20240713d/s2_sr_2019_2023_gMedian_AK050H60V10_all_v20240713d.tif
gs://akveg-data/s2_sr_2019_2023_gMedian_v20240713d/s2_sr_2019_2023_gMedian_AK050H60V11_all_v20240713d.tif
gs://akveg-data/s2_sr_2019_2023_gMedian_v20240713d/s2_sr_2019_2023_gMedian_AK050H60V12_all_v20240713d.tif
gs://akveg-data/s2_sr_2019_2023_gMedian_v20240713d/s2_sr_2019_2023_gMedian_AK050H60V13_all_v20240713d.tif
gs://akveg-data/s2_sr_2019_2023_gMedian_v20240713d/s2_sr_2019_2023_gMedian_AK050H60V14_all_v20240713d.tif
gs://akveg-data/s2_sr_2019_2023_gMedian_v20240713d/s2_sr_2019_2023_gMedian_AK050H60V15_all_v20240713d.tif
gs://akveg-data/s2_sr_2019_2023_gMedian_v20240713d/s2_sr_2019_2023_gMedian_AK050H60V16_all_v20240713d.tif
gs://akveg-data/s2_sr_2019_2023_gMedian_v20240713d/s2_sr_2019_2023_gMedian_AK050H60V17_all_v20240713d.tif
gs://akveg-data/s2_sr_2019_2023_gMedian_v20240713d/s2_sr_2019_2023_gMedian_AK050H60V18_all_v20240713d.tif
gs://akveg-data/s2_sr_2019_2023_gMedian_v20240

## Function to load list of gcs cogs to GEE imageCollection
comment out pprint and most print except when troubleshooting

In [36]:
def load_gcs_cogs_to_collection(cogs, project_folder, collection):
    # Request body as a dictionary.
    for cog in cogs['tif']:
      fileOnly = os.path.split(cog)[1]
      # print(fileOnly)

      cogName = fileOnly[:-4]
      print(cogName)
      
      parts = fileOnly.split('_')
      # print(parts)
      
      season = parts[6]
      # print(season)
        
      request = {
        'type': 'IMAGE',
        'gcs_location': {
          'uris': cog
        },
        'properties': {
        #   'source': 'https://code.earthengine.google.com/067b10ee56537817756a3177a9138aee',
            'seasonName': season
        },
        'startTime': '2023-01-01T00:00:00.000000000Z',
        'endTime': '2024-01-01T00:00:00.000000000Z',
      }

      # pprint(json.dumps(request))

      # A folder (or ImageCollection) name and the new asset name.
      asset_id = collection+'/'+cogName
      # print(project_folder)
      # print(asset_id)
        
      url = 'https://earthengine.googleapis.com/v1alpha/projects/{}/assets?assetId={}'
      # print(url)

      response = session.post(
        url = url.format(project_folder, asset_id),
        data = json.dumps(request)
      )

      # pprint(json.loads(response.content))
    print('done')


## Run it

In [37]:
# load_gcs_cogs_to_collection(cogs, project_folder, 's1_2022_v20230326')
# load_gcs_cogs_to_collection(cogs, project_folder, 's2_sr_2019_2023_gMedian_v20240710')
load_gcs_cogs_to_collection(cogs, project_folder, 's2_sr_2019_2023_gMedian_v20240713d')


s2_sr_2019_2023_gMedian_AK050H60V10_all_v20240713d
s2_sr_2019_2023_gMedian_AK050H60V11_all_v20240713d
s2_sr_2019_2023_gMedian_AK050H60V12_all_v20240713d
s2_sr_2019_2023_gMedian_AK050H60V13_all_v20240713d
s2_sr_2019_2023_gMedian_AK050H60V14_all_v20240713d
s2_sr_2019_2023_gMedian_AK050H60V15_all_v20240713d
s2_sr_2019_2023_gMedian_AK050H60V16_all_v20240713d
s2_sr_2019_2023_gMedian_AK050H60V17_all_v20240713d


ReadTimeout: HTTPSConnectionPool(host='earthengine.googleapis.com', port=443): Read timed out. (read timeout=120)

## Cleanup existing imageCollection, if needed

imageCollections cannot be deleted until all images inside them are deleted. For a cloud-backed image collection with hundreds or thousands of tiles, this can take a while

bash earthengine CLI
```
for i in `earthengine ls projects/akveg-map/assets/s2_sr_2019_2023_gMedian_v20240713d`; do earthengine rm $i; done
# earthengine rm projects/akveg-map/assets/s2_2019_2023_gMedian_v20240311
```